In [ ]:
import os
import math
import mxnet as mx
from mxnet import image
from mxnet import nd, gluon, autograd, init
from mxnet.gluon import nn
from tensorboardX import SummaryWriter
import numpy as np
import shutil
from mxnet.gluon.data import vision
import pandas as pd

In [ ]:
%run resnet.py

In [ ]:
demo = False
if demo:
    # 注意：此处使用小训练集为便于网页编译。Kaggle的完整数据集应包括5万训练样本。
    train_dir = 'train_tiny'
    # 注意：此处使用小测试集为便于网页编译。Kaggle的完整数据集应包括30万测试样本。
    test_dir = 'test_tiny'
    # 注意：此处相应使用小批量。对Kaggle的完整数据集可设较大的整数，例如128。
    batch_size = 1
else:
    train_dir = 'train'
    test_dir = 'test'
    batch_size = 128

data_dir = '../data/kaggle_cifr-10'
label_file = 'trainLabels.csv'
input_dir = 'train_valid_test'
valid_ratio = 0.1

In [ ]:
def reorg_cifar10_data(data_dir, label_file, train_dir, test_dir, input_dir, valid_ratio):
    # 读取训练数据标签。
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）。
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((int(idx), label) for idx, label in tokens))
    labels = set(idx_label.values())

    num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
    num_train_tuning = int(num_train * (1 - valid_ratio))
    assert 0 < num_train_tuning < num_train
    num_train_tuning_per_label = num_train_tuning // len(labels)
    label_count = dict()

    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))

    # 整理训练和验证集。
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = int(train_file.split('.')[0])
        label = idx_label[idx]
        mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < num_train_tuning_per_label:
            mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))

    # 整理测试集。
    mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))
        


In [ ]:
first_run = False
if first_run:
    reorg_cifar10_data(data_dir, label_file, train_dir, test_dir, input_dir, valid_ratio)

In [ ]:
def transform_train(data, label):
    im = data.asnumpy()
    im = np.pad(im, ((4, 4), (4, 4), (0, 0)), mode='constant', constant_values=0)
    im = nd.array(im, dtype='float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, rand_mirror=True,
                                    rand_crop=True,
                                   mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1)) # channel x width x height
    return im, nd.array([label]).astype('float32')

def transform_test(data, label):
    im = data.astype('float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1))
    return im, nd.array([label]).astype('float32')

In [ ]:
input_str = data_dir + '/' + input_dir + '/'

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + 'train', flag=1, 
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag=1, 
                                     transform=transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid', 
                                           flag=1, transform=transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1, 
                                     transform=transform_test)

loader = gluon.data.DataLoader

train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle=True, last_batch='keep')
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')

In [ ]:
import datetime
writer = SummaryWriter()

criterion = gluon.loss.SoftmaxCrossEntropyLoss()

def get_acc(output, label):
    pred = output.argmax(1, keepdims=True)
    correct = (pred == label).sum()
    return correct.asscalar()

def train(net, train_data, valid_data, start_epoch, num_epochs, lr, wd, ctx, lr_decay):
    print("train called")
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9, 'wd': wd})

    prev_time = datetime.datetime.now()
    for epoch in range(start_epoch, start_epoch + num_epochs):
        train_loss = 0
        correct = 0
        total = 0
        if epoch == 89 or epoch == 139:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            bs = data.shape[0]
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data)
                loss = criterion(output, label)
            loss.backward()
            trainer.step(bs)
            train_loss += nd.mean(loss).asscalar()
            correct += get_acc(output, label)
            total += bs
        train_acc = correct / total
        writer.add_scalars('loss', {'train': train_loss / len(train_data)}, epoch)
        writer.add_scalars('acc', {'train': correct / total}, epoch)
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_correct = 0
            valid_total = 0
            valid_loss = 0
            for data, label in valid_data:
                bs = data.shape[0]
                data = data.as_in_context(ctx)
                label = label.as_in_context(ctx)
                output = net(data)
                loss = criterion(output, label)
                valid_loss += nd.mean(loss).asscalar()
                valid_correct += get_acc(output, label)
                valid_total += bs
            valid_acc = valid_correct / valid_total
            writer.add_scalars('loss', {'valid': valid_loss / len(valid_data)}, epoch)
            writer.add_scalars('acc', {'valid': valid_acc}, epoch)
            epoch_str = ("Epoch %d. Train Loss: %f, Train acc %f, Valid Loss: %f, Valid acc %f, "
                         % (epoch, train_loss / len(train_data),
                            train_acc, valid_loss / len(valid_data), valid_acc))
        else:
            epoch_str = ("Epoch %d. Loss: %f, Train acc %f, "
                         % (epoch, train_loss / len(train_data),
                            correct / total))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))
        if (epoch % 2 == 0):
            filename = "./resnet164v2-%s-v-%s-t-%s-lr-%s-wd-%s.params" % (epoch, valid_acc, train_acc, trainer.learning_rate, wd)
            net.save_params(filename)
    return train_acc, epoch

In [ ]:
def get_net(ctx):
    num_outputs = 10
    net = ResNet164_v2(num_outputs)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net

ctx = mx.gpu(0)
net = get_net(ctx)
net.hybridize()

In [ ]:
# Resnet for training
start_epoch = 0
num_epochs = 200
#learning_rate = 0.0001
learning_rate = 0.00001
#weight_decay = 1e-4
weight_decay = 1e-5
lr_decay = 0.1
train_acc, end_epoch = train(net, train_data, valid_data, start_epoch, num_epochs, learning_rate, weight_decay, ctx, lr_decay)

In [ ]:
# Resnet train all dataset for submission
start_epoch= end_epoch
learning_rate = 0.00005
weight_decay = 5e-4
lr_decay=0.1
num_epochs=1
train_acc = train(net, train_valid_data, None, start_epoch, num_epochs, learning_rate, weight_decay, ctx, lr_decay)
print("train acc", train_acc)

In [ ]:
# Load existing models
#net = get_net(mx.gpu(0))
#net.hybridize()
#net.load_params('./resnet164v2-188.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-230.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-274.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-364-0.954091816367-0.999844461726-5e-05-0.0005.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-282.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-324.params', ctx=mx.gpu(0))

In [ ]:
# predict with one model 
preds=[]
for data, label in test_data:
    output = net(data.as_in_context(mx.gpu(0)))
    preds.extend(output.argmax(axis=1).astype(int).asnumpy())
print('Done test')
sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_valid_ds.synsets[x])
df.to_csv('submission-resnet-%s.csv' % train_acc, index=False)
print("Done submission")

In [ ]:
# Save model
kaggle_score="0.95440"
filename = "./kaggel-resnet-%s.params" % kaggle_score
net.save_params(filename)

In [ ]:
# Load existing models
#net = get_net(mx.gpu(0))
#net.hybridize()
#net.load_params('./resnet164v2-188.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-230.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-274.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-364-0.954091816367-0.999844461726-5e-05-0.0005.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-282.params', ctx=mx.gpu(0))
#net.load_params('./resnet164v2-324.params', ctx=mx.gpu(0))

# Load pretrained models
#from mxnet.gluon.model_zoo import vision as models
#pretrained_net = models.resnet18_v2(pretrained=True)
#weight = pretrained_net.classifier[4].params.get('weight')